In [5]:
import os
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [8]:
import math
def load_to_dataframe(train_folder_path, test_folder_path):
    """
    Loads train and test data from the specified file folders, and returns them as pandas DataFrame objects.
    :param train_folder_path: The path to the folder containing the train data files.
    :type train_folder_path: str
    :param test_folder_path: The path to the folder containing the test data files.
    :type test_folder_path: str
    :return: A two pandas DataFrame objects, containing the train and test data, respectively.
    :rtype: pandas.core.frame.DataFrame, pandas.core.frame.DataFrame
    """
    train_path = 'dataset/train'
    test_path = 'dataset/test'
    train_dir_list = os.listdir(train_path)
    test_dir_list = os.listdir(test_path)

    train_sentiment_value = []
    test_sentiment_value = []
    train_file_path = []
    test_file_path = []

    for file in train_dir_list:
        train_file_path.append(train_path + '/' + file)
        sentiment_code = file.split('_')
        if sentiment_code[2] == 'ANG':
            train_sentiment_value.append('angry')
        elif sentiment_code[2] == 'DIS':
            train_sentiment_value.append('disgust')
        elif sentiment_code[2] == 'FEA':
            train_sentiment_value.append('fear')
        elif sentiment_code[2] == 'HAP':
            train_sentiment_value.append('happy')
        elif sentiment_code[2] == 'NEU':
            train_sentiment_value.append('neutral')
        elif sentiment_code[2] == 'SAD':
            train_sentiment_value.append('sad')
        else:
            train_sentiment_value.append('unknown')

    for file in test_dir_list:
        test_file_path.append(test_path + '/' + file)
        sentiment_code = file.split('_')
        if sentiment_code[2] == 'ANG':
            test_sentiment_value.append('angry')
        elif sentiment_code[2] == 'DIS':
            test_sentiment_value.append('disgust')
        elif sentiment_code[2] == 'FEA':
            test_sentiment_value.append('fear')
        elif sentiment_code[2] == 'HAP':
            test_sentiment_value.append('happy')
        elif sentiment_code[2] == 'NEU':
            test_sentiment_value.append('neutral')
        elif sentiment_code[2] == 'SAD':
            test_sentiment_value.append('sad')
        else:
            test_sentiment_value.append('unknown')

    train_sentiment_df = pd.DataFrame(
        {"File_Path": train_file_path, "Target": train_sentiment_value})

    test_sentiment_df = pd.DataFrame(
        {"File_Path": test_file_path, "Target": test_sentiment_value})

    return train_sentiment_df, test_sentiment_df

def turn_into_data_for_model(train_df, test_df, mfcc_number=40, fft=2048, hop=512, AugmentType = ""):
    # Set Variable for MFCC
    num_mfcc = mfcc_number
    SAMPLE_RATE = 16000
    n_fft = fft
    hop_length = hop

    train_data = {
        "labels": [],
        "mfcc": []
    }

    test_data = {
        "labels": [],
        "mfcc": []
    }

    # Encode Categories
    labels = {'disgust': 0, 'happy': 1, 'sad': 2,
              'neutral': 3, 'fear': 4, 'angry': 5}
    train_df_encoded = train_df.replace({'Target': labels}, inplace=False)
    test_df_encoded = test_df.replace({'Target': labels}, inplace=False)

    for item, row in train_df.iterrows():
        train_data['labels'].append(train_df_encoded.iloc[item, 1])
        signal, sample_rate = librosa.load(
            train_df_encoded.iloc[item, 0], sr=SAMPLE_RATE)
        mfcc = librosa.feature.mfcc(
            y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
        mfcc = mfcc.T
        train_data["mfcc"].append(np.asarray(mfcc))

        if AugmentType != '':
            try:
                newAugment = train_df_encoded.iloc[item, 0].replace("train", "Augmented/{}".format(AugmentType))
                signal, sample_rate = librosa.load(
                    newAugment, sr=SAMPLE_RATE)
                mfcc = librosa.feature.mfcc(
                    y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                mfcc = mfcc.T
                train_data['labels'].append(train_df_encoded.iloc[item, 1])
                train_data["mfcc"].append(np.asarray(mfcc))
            except:
                print(newAugment + "Not Found")

        if item % 300 == 0:
            print("Train Size:" + str(math.floor(item)))

    # for item, row in test_df.iterrows():
    #     test_data['labels'].append(test_df_encoded.iloc[item, 1])
    #     signal, sample_rate = librosa.load(
    #         test_df_encoded.iloc[item, 0], sr=SAMPLE_RATE)
    #     mfcc = librosa.feature.mfcc(
    #         y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    #     mfcc = mfcc.T
    #     test_data["mfcc"].append(np.asarray(mfcc))
    #     if item % 300 == 0:
    #         print("Test Size:" + str(math.floor(item)))

    train_data_value = np.asarray(train_data['mfcc'])
    train_data_target = np.asarray(train_data["labels"])
    # train_data_value = tf.keras.preprocessing.sequence.pad_sequences(
    #     train_data_value, maxlen=156)
    # test_data_value = np.asarray(test_data['mfcc'])
    # test_data_target = np.asarray(test_data["labels"])
    # test_data_value = tf.keras.preprocessing.sequence.pad_sequences(
    #     test_data_value, maxlen=156)
    print(train_data_value.shape)
    # print(test_data_value.shape)

    # return train_data_value, train_data_target, test_data_value, test_data_target
    return train_data_value, train_data_target

In [10]:
train_df, test_df = load_to_dataframe('dataset/train', 'dataset/test')


In [11]:
crema_train_gaussian_value, crema_train_gaussian_target = turn_into_data_for_model(
    train_df, test_df, 40, 2048, 512, AugmentType="AddGaussianNoise")

Train Size:0
Train Size:300
Train Size:600
Train Size:900
Train Size:1200
Train Size:1500
Train Size:1800
Train Size:2100
Train Size:2400
Train Size:2700
Train Size:3000
Train Size:3300
Train Size:3600
Train Size:3900
Train Size:4200
Train Size:4500
Train Size:4800
Train Size:5100
Train Size:5400
Train Size:5700


e:\TA-Bill\venv\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/Augmented/AddGaussianNoise/1091_WSI_SAD_XX.wav'

In [ ]:
crema_train_pitch_value, crema_train_pitch_target = turn_into_data_for_model(
    train_df, test_df, 40, 2048, 512, AugmentType="PitchShift")

In [ ]:
crema_train_timeStretch_value, crema_train_timeStretch_target = turn_into_data_for_model(
    train_df, test_df, 40, 2048, 512, AugmentType="TimeStretch")